In [1]:
import pandas as pd
import numpy as np
import math
import datetime
from ipynb.fs.defs.Local_search import new_route
from ipynb.fs.defs.Local_search import get_row
from ipynb.fs.defs.Local_search import get_column
from ipynb.fs.defs.Local_search import best_index
from ipynb.fs.defs.Local_search import schedule_time
from ipynb.fs.defs.Local_search import find_best_position
from ipynb.fs.defs.Local_search import cities_list
#table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
#time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
#dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
#fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
#fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1")
#fur1d = pd.read_csv("../Dati/turni_f/furgoncino1_KM_routing.csv", delimiter=";", encoding = "ISO-8859-1")
#fur2d = pd.read_csv("../Dati/turni_f/furgoncino2_KM_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur1 = pd.read_csv("../Dati/turni_f/2-opt1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur2 = pd.read_csv("../Dati/turni_f/2-opt2_shifts.csv", delimiter=";", encoding = "ISO-8859-1")

In [2]:
#Given a list of cities it return False if "to_insert" is already present
#Return True if "to_insert" isn't present and it will give the index of the best position
def check_insert(schedule_day_list, to_insert, dframe):
    if to_insert in schedule_day_list:
        return False, -1
    else:
        #print(schedule_day_list)
        schedule_day_list = cities_list(new_route(schedule_day_list, dframe))
        to_return = best_index(schedule_day_list, new_route(schedule_day_list, dframe), to_insert, dframe)
        return True, to_return

#Given: city list wich to_insert come, the city "to_insert" to be moved, 
#the city list where "to_insert" will be insered, the index of the best position and a dataframe
#the function return the improvement
#If negative the exchange mustn't be done, if positive you gain some time
def improvement(f_l, to_insert, t_l, index, dframe):
    from_list = f_l[:]
    from_time = sum(new_route(from_list, dframe).values)
    from_list.remove(to_insert)
    from_time_removed = sum(new_route(from_list, dframe).values)
    
    to_list = t_l[:]
    to_time = sum(new_route(to_list, dframe).values)
    to_list.insert(index, to_insert)
    to_time_insert = sum(new_route(to_list, dframe).values)
    
    return (from_time-from_time_removed) + (to_time-to_time_insert)

In [3]:
l = list(fur1["lunedi"].dropna())
m = list(fur1["martedi"].dropna())
i = l[3]
b, pos = check_insert(m, i, dft)
if b:
    print(improvement(l, i, m, pos,dft))

11.0


In [39]:
def get_best_improvement(curr_day, veichle, best_improvement, best_day, best_veichle, best_idx, to_insert, curr_f_list):
    if(veichle==1):
        days = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
    else:
        days = ["martedi", "mercoledi", "giovedi"]
        
    for inner_day in days:
        if(veichle==1):
            curr_to_list = list(fur1[inner_day].dropna())
        else:
            curr_to_list = list(fur2[inner_day].dropna())
           
        #print(inner_day)
        if inner_day == curr_day:
            #print("continua")
            continue
        else:
            boo, index = check_insert(curr_to_list, to_insert, dft)

        if boo:
            local_improvement = improvement(curr_f_list, to_insert, curr_to_list, index, dft)
        else:
            continue

        #If there's an improvement we update "best" variables, in other case we skip to next day
        #print(local_improvement)
        if local_improvement<0:
            continue
        elif local_improvement>best_improvement:
            best_improvement = local_improvement
            best_day = inner_day
            best_veichle = veichle
            best_idx = index
        else:
            continue
    return best_improvement, best_day, best_veichle, best_idx

In [40]:
get_best_improvement("lunedi", 1, -1000, "", "", "", fur1["lunedi"][4], list(fur1["lunedi"].dropna()))

(7.0, 'mercoledi', 1, 4)

In [46]:
fur1

,lunedi,martedi,mercoledi,giovedi,venerdi
0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,"Sesto ed Uniti, Roma, 4","Cremona, Ugolani Dati, 4",Castelverde,"Cremona, Ugolani Dati, 4","Crema, Civerchi, 9"
2,"Grumello Cremonese ed Uniti, Roma, 2","Casalpusterlengo, Pusterla 7",Annicco,"Persico Dosimo, Albert ,7","Casaletto Vaprio, Maggiore, 7"
3,"Pizzighettone, Giuseppe Garibaldi, 18","Vailate, Dante Alighieri, 7",Castelleone,"Grontardo, Roma, 16","Trescore Cremasco, Vittorio Veneto, 1"
4,"Castelleone, Roma, 67","Crema, Civerchi, 9",Madignano,"Vescovato, Roma, 44","Pandino, Castello, 15"
5,"Montodine, XXV Aprile, 10","Persico Dosimo, Albert ,7",Crema,"Crema, Civerchi, 9","Agnadello, Marconi, 6"
6,"Ripalta Cremasca, Roma, 5","Scandolara Ravara, Italia",Palazzo Pignano,"Offanengo, Clavelli Martini, 5","Rivolta d'Adda, Stefano Renzi, 5"
7,"Ricengo, Guglielmo Marconi","Casalmaggiore, Guglielmo Marconi, 8",Pandino,"Izano, Gerardo da Izano, 4/C","Spino d'Adda, Circonvallazione, 13"
8,"Capralba, Piave, 2","San Giovanni in Croce, Feudatari, 25 A",Rivolta D'adda,"Manerbio, INSURREZIONE NAZIONALE","Dovera, Oldrini 2"
9,"Sergnano, IV Novembre, 10","Ostiano, Mazzini, 1",Spino d'adda,"Casalmaggiore, Guglielmo Marconi, 8","Monte Cremasco, Roma, 12"


In [55]:
#For every day of every veichle we get every Location
#We compare the improvement removing this location from the day/veichle routing list and inserting it 
#in the other day/veichle routing lists.
#The location will be moved only if the improvement is positive 
#In case of different positive improvement we move in the higher
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
short_week = ["martedi", "mercoledi", "giovedi"]
fur1_copy = pd.DataFrame({})
fur1_copy['col'] = pd.Series([0]*50)
fur1_copy = fur1_copy.join(fur1)

fur2_copy = pd.DataFrame({})
fur2_copy['col'] = pd.Series([0]*50)
fur2_copy = fur2_copy.join(fur2)
#for day in week:
day="lunedi"
print(day, end="\n")
curr_from_list = list(fur1_copy[day].dropna())
to_insert_list = list(fur1_copy[day].dropna())
cycles = len(to_insert_list)
#For every location in this schedule
for i in range(1, cycles-1):
    #print(to_insert_list)
    to_insert = to_insert_list.pop(0)
    #print(to_insert, end=", ")
    #to_insert = to_insert_list.pop(0)

    best_improvement = -1000
    best_day = ""
    best_veichle = ""
    best_idx = ""
    #Here we must compare every possible insert in other days
    #Cycle for first veichle
    best_improvement, best_day, best_veichle, best_idx = get_best_improvement(day, 1, best_improvement, best_day, best_veichle, best_idx, to_insert, curr_from_list)
    best_improvement, best_day, best_veichle, best_idx = get_best_improvement(day, 2, best_improvement, best_day, best_veichle, best_idx, to_insert, curr_from_list)
    #If best_day isn't modified where isn't an improvement for every day/veichle
    if(best_day==""):
        continue
    else:
        
        #Remove from origin list and insert into target list
        curr_from_list.remove(to_insert)
        if(best_veichle==1):
            print(f"{to_insert} miglioramento veicolo 1 a {best_day}")
            temp_list = list(fur1_copy[best_day].dropna())
            temp_list.insert(best_idx, to_insert)
            padding = pd.Series([np.nan]*(50-len(temp_list)))
            temp_list = pd.concat([pd.Series(temp_list), padding], ignore_index=True)
            fur1_copy[best_day] = fur1_copy[best_day].replace(list(fur1_copy[best_day]), temp_list)
        elif(best_veichle==2):
            print(f"{to_insert} miglioramento veicolo 2 a {best_day}")
            temp_list = list(fur2_copy[best_day].dropna())
            temp_list.insert(best_idx, to_insert)
            padding = pd.Series([np.nan]*(50-len(temp_list)))
            temp_list = pd.concat([pd.Series(temp_list), padding], ignore_index=True)
            fur2_copy[best_day] =  fur2_copy[best_day].replace(list(fur2_copy[best_day]), temp_list)
fur1_copy   

lunedi
Pizzighettone, Giuseppe Garibaldi, 18 miglioramento veicolo 1 a martedi
Castelleone, Roma, 67 miglioramento veicolo 1 a mercoledi
Ricengo, Guglielmo Marconi miglioramento veicolo 2 a martedi
Capralba, Piave, 2 miglioramento veicolo 1 a martedi
Sergnano, IV Novembre,  10 miglioramento veicolo 2 a martedi
Trigolo, Roma, 26 miglioramento veicolo 2 a mercoledi


,col,lunedi,martedi,mercoledi,giovedi,venerdi
0,0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,0,"Sesto ed Uniti, Roma, 4","Cremona, Ugolani Dati, 4",Castelverde,"Cremona, Ugolani Dati, 4","Crema, Civerchi, 9"
2,0,"Grumello Cremonese ed Uniti, Roma, 2","Pizzighettone, Giuseppe Garibaldi, 18",Annicco,"Persico Dosimo, Albert ,7","Casaletto Vaprio, Maggiore, 7"
3,0,"Pizzighettone, Giuseppe Garibaldi, 18","Casalpusterlengo, Pusterla 7",Castelleone,"Grontardo, Roma, 16","Trescore Cremasco, Vittorio Veneto, 1"
4,0,"Castelleone, Roma, 67","Capralba, Piave, 2","Castelleone, Roma, 67","Vescovato, Roma, 44","Pandino, Castello, 15"
5,0,"Montodine, XXV Aprile, 10","Vailate, Dante Alighieri, 7",Madignano,"Crema, Civerchi, 9","Agnadello, Marconi, 6"
6,0,"Ripalta Cremasca, Roma, 5","Crema, Civerchi, 9",Crema,"Offanengo, Clavelli Martini, 5","Rivolta d'Adda, Stefano Renzi, 5"
7,0,"Ricengo, Guglielmo Marconi","Persico Dosimo, Albert ,7",Palazzo Pignano,"Izano, Gerardo da Izano, 4/C","Spino d'Adda, Circonvallazione, 13"
8,0,"Capralba, Piave, 2","Scandolara Ravara, Italia",Pandino,"Manerbio, INSURREZIONE NAZIONALE","Dovera, Oldrini 2"
9,0,"Sergnano, IV Novembre, 10","Casalmaggiore, Guglielmo Marconi, 8",Rivolta D'adda,"Casalmaggiore, Guglielmo Marconi, 8","Monte Cremasco, Roma, 12"


In [16]:
fur2_copy = pd.DataFrame({})
fur2_copy['col'] = pd.Series([np.nan]*50)
fur2_copy = fur2_copy.join(fur2)
fur2_copy[["martedi", "mercoledi", "giovedi"]]

,martedi,mercoledi,giovedi
0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,"Cremona, Felice Cavallotti, 2","Cremona, Palosca n. 2","Cremona, Ugolani Dati, 4"
2,"Pianengo, Nenni","Paderno Ponchielli, Municipio, 3","Pizzighettone, Giuseppe Garibaldi, 18"
3,"Offanengo, Clavelli Martini, 5","San Bassano, Mons. Frosi",Gadesco
4,"Manerbio, INSURREZIONE NAZIONALE","Fiesco, Giuseppe Parini, 10","Pescarolo ed Uniti, Giuseppe Mazzini, 73"
5,"Bonemerse, Roma, 25","Soncino, Dante Alighieri, 23","Cingia de' Botti, Giuseppina, 79"
6,"Sospiro, Tosi","Soresina, Giacomo Matteotti, 6","Martignana di Po, LibertÃ , 64"
7,Gadesco,"Casalbuttano ed Uniti, Jacini, 23","Solarolo Rainerio, Giuseppina, 81"
8,"Vescovato, Roma, 44","San Giovanni in Croce, Feudatari, 25 A","San Giovanni in Croce, Feudatari, 25 A"
9,"Pescarolo ed Uniti, Giuseppe Mazzini, 73",NaN,NaN


In [67]:
a=pd.Series(["a","b","c"])
b=pd.Series([np.nan]*(10-len(a)))
pd.concat([a,b], ignore_index=True)

0      a
1      b
2      c
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
dtype: object

In [51]:
fur1_copy = pd.DataFrame({})
fur1_copy['col'] = pd.Series([0]*50)
fur1_copy = fur1_copy.join(fur1)
temp_list = pd.Series(["ciao"])
padding=pd.Series([np.nan]*(50-len(temp_list)))
temp_list = pd.concat([temp_list, padding], ignore_index=True)
fur1_copy[best_day] = fur1_copy[best_day].replace(list(fur1_copy[best_day]), temp_list)